In [ ]:
# IMPORT LIBRARIES
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset, TensorDataset
import urllib
from torch.optim import *
import torchvision
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from torch.autograd import Variable
from torch.optim import lr_scheduler
import copy
import time
import os


plt.ion()   # interactive mode
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
import os
import sys
from google.colab import drive
drive.mount('/content/gdrive/')

data_file_location = "/content/gdrive/MyDrive/MNIST/"
# data_file_location = "data_bn"
os.chdir(data_file_location)
# # sys.path.append(os.path.abspath(py_file_location))
print(os.getcwd())

Mounted at /content/gdrive/
/content/gdrive/MyDrive/MNIST


In [ ]:
mnist_data = torchvision.datasets.MNIST('/content/gdrive/MyDrive/MNIST/')

In [ ]:

dl = torch.utils.data.DataLoader(mnist_data, batch_size=16, shuffle=False)

In [ ]:
#Pre-processing of Dataset
tensor = dl.dataset.data
tr = tensor.reshape(tensor.size(0), -1)  # reshaped
targets = dl.dataset.targets

In [ ]:
#---------------------------------Unseen Data : 5000-----------------------
#Unseen Data : 5000

mnist_unseen_data = tr[0:5000]
mnist_unseen_data_final = mnist_unseen_data.view(-1, 1,28,28).float()
mnist_unseen_label = targets[0:5000]

prob_ds = TensorDataset(mnist_unseen_data_final, mnist_unseen_label)

prob_dl = torch.utils.data.DataLoader(prob_ds, batch_size=1, shuffle=False)

In [ ]:
#Training Data: 44000

trainset_data = tr[5000: 49000]
trainset_target = targets[5000: 49000]

trainset_data_numpy = np.array(trainset_data)
trainset_target_numpy = np.array(trainset_target)

train_with_target = np.c_[trainset_data_numpy, trainset_target_numpy]

In [ ]:
# Test Data: 11000

test_data = tr[49000:]
test_data_final = test_data.view(-1, 1,28,28).float()
test_target = targets[49000:]

#Test Set
test_ds = TensorDataset(test_data_final, test_target)

#Test Loader
test_dl = torch.utils.data.DataLoader(test_ds, batch_size=1, shuffle=True)

In [ ]:
#Trainset A:

#A: Data
trainset_a = train_with_target[0: 17500]
trainset_a_pd = pd.DataFrame(trainset_a)

#Common Data of 25%, 50% and 75%
chunk_a1 = trainset_a_pd.sample(4375) #25% Common data
chunk_a2 = trainset_a_pd.sample(8750) #50% Common data
chunk_a3 = trainset_a_pd.sample(13125) #75% Common data


chunk_a1 = pd.DataFrame(np.array(chunk_a1))
chunk_a2 = pd.DataFrame(np.array(chunk_a2))
chunk_a3 = pd.DataFrame(np.array(chunk_a3))

In [ ]:
#Trainset B: 25% Common of A

#B: Data
trainset_b = train_with_target[17500: 30625] #13125 data
trainset_b_pd = pd.DataFrame(trainset_b)
trainset_b_pd = pd.DataFrame(np.array(trainset_b_pd))
trainset_b_pd = trainset_b_pd.append(chunk_a1) #4375
trainset_b_pd = pd.DataFrame(np.array(trainset_b_pd))


In [ ]:
#Trainset C: 50% common from A

#C: Data
trainset_c = train_with_target[30625: 39375] #8750 data
trainset_c_pd = pd.DataFrame(trainset_c)
trainset_c_pd = pd.DataFrame(np.array(trainset_c_pd))
trainset_c_pd = trainset_c_pd.append(chunk_a2) #8750
trainset_c_pd = pd.DataFrame(np.array(trainset_c_pd))

In [ ]:
#Trainset d: 75% common from A

#D: Data
trainset_d = train_with_target[39375: 43750] #4375 data
trainset_d_pd = pd.DataFrame(trainset_d)
trainset_d_pd = pd.DataFrame(np.array(trainset_d_pd))
trainset_d_pd = trainset_d_pd.append(chunk_a3)
trainset_d_pd = pd.DataFrame(np.array(trainset_d_pd))

In [ ]:
#Trainset E: 100% common from A

#E: Data
trainset_e = train_with_target[0: 17500]
trainset_e_pd = pd.DataFrame(trainset_e)

In [ ]:
#Batch size
bs = 64

In [ ]:
#Trainset A Data and Target to Tensor
init_train_a = trainset_a_pd.iloc[:, : 784].values
train_a = torch.tensor(init_train_a)
train_a_final = train_a.view(-1, 1,28,28).float()

init_train_a_target = trainset_a_pd.iloc[:, -1].values
train_a_target = torch.tensor(init_train_a_target)


#Trainset A Data and Target Tensor to Dataset then Dataloader
train_v_ds = TensorDataset(train_a_final, train_a_target)
#Tensor Dataset train v to Dataloader
# Trainset 1: train_v_dl
train_v_dl = torch.utils.data.DataLoader(train_v_ds, batch_size=bs, shuffle=True)

In [ ]:
#Trainset B Data and Target to Tensor
init_train_b = trainset_b_pd.iloc[:, : 784].values
train_b = torch.tensor(init_train_b)
train_b_final = train_b.view(-1, 1,28,28).float()

init_train_b_target = trainset_b_pd.iloc[:, -1].values
train_b_target = torch.tensor(init_train_b_target)


#Trainset A Data and Target Tensor to Dataset then Dataloader
train_w_ds = TensorDataset(train_b_final, train_b_target)
#Tensor Dataset train v to Dataloader
# Trainset 2: train_w_dl
train_w_dl = torch.utils.data.DataLoader(train_w_ds, batch_size=bs, shuffle=True)

In [ ]:
#Trainset C Data and Target to Tensor
init_train_c = trainset_c_pd.iloc[:, : 784].values
train_c = torch.tensor(init_train_c)
train_c_final = train_c.view(-1, 1,28,28).float()

init_train_c_target = trainset_c_pd.iloc[:, -1].values
train_c_target = torch.tensor(init_train_c_target)


#Trainset C Data and Target Tensor to Dataset then Dataloader
train_x_ds = TensorDataset(train_c_final, train_c_target)
#Tensor Dataset train x to Dataloader
# Trainset 3: train_x_dl
train_x_dl = torch.utils.data.DataLoader(train_x_ds, batch_size=bs, shuffle=True)

In [ ]:
#Trainset D Data and Target to Tensor
init_train_d = trainset_d_pd.iloc[:, : 784].values
train_d = torch.tensor(init_train_d)
train_d_final = train_d.view(-1, 1,28,28).float()

init_train_d_target = trainset_d_pd.iloc[:, -1].values
train_d_target = torch.tensor(init_train_d_target)


#Trainset D Data and Target Tensor to Dataset then Dataloader
train_y_ds = TensorDataset(train_d_final, train_d_target)
#Tensor Dataset train y to Dataloader
# Trainset 4: train_y_dl
train_y_dl = torch.utils.data.DataLoader(train_y_ds, batch_size=bs, shuffle=True)

In [ ]:
#Trainset E Data and Target to Tensor
init_train_e = trainset_e_pd.iloc[:, : 784].values
train_e = torch.tensor(init_train_e)
train_e_final = train_e.view(-1, 1,28,28).float()

init_train_e_target = trainset_e_pd.iloc[:, -1].values
train_e_target = torch.tensor(init_train_e_target)


#Trainset E Data and Target Tensor to Dataset then Dataloader
train_z_ds = TensorDataset(train_e_final, train_e_target)
#Tensor Dataset train z to Dataloader
# Trainset 4: train_z_dl
train_z_dl = torch.utils.data.DataLoader(train_z_ds, batch_size=bs, shuffle=True)

In [ ]:
###-------------- Define a Convolutional Neural Network ---------------###

#-------------------------------------------------------------------------

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)


In [ ]:
train_size = len(trainset_a)
test_size = len(test_ds)

print(train_size, test_size)

17500 11000


In [ ]:
def train_model(model, train_loader, test_loader, criterion, optimizer, scheduler, num_epochs=20):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            
            if phase == 'train':
                model.train()  # Set model to training mode
                
                running_loss = 0.0
                running_corrects = 0
                
                for (idx, (inputs, labels)) in enumerate(train_loader):
                    inputs = inputs.to(device)
                    labels = labels.to(device)
                    
                    
                    optimizer.zero_grad()
                    with torch.set_grad_enabled(True):
                        outputs = model(inputs)
                        
                        _, preds = torch.max(outputs, 1)
                        # print(preds, labels)
                        loss = criterion(outputs, labels)
                        
                        loss.backward()
                        optimizer.step()
                    
                    running_loss += loss.item() * inputs.size(0)
                    running_corrects += torch.sum(preds == labels.data)
                    
                    
                scheduler.step()
                print(running_corrects)
                epoch_loss = running_loss / train_size
                epoch_acc = running_corrects.double() / train_size

                print('{} Loss: {:.4f} Acc: {:.4f}'.format('Train', epoch_loss, epoch_acc))
                    
            else:
                
                model.eval()   # Set model to evaluate mode
                
                running_loss = 0.0
                running_corrects = 0
                
                for data in test_loader:
                    inputs, labels = data
                    inputs = inputs.to(device)
                    labels = labels.to(device)
                    optimizer.zero_grad()
                    with torch.set_grad_enabled(False):
                        outputs = model(inputs)
                        _, preds = torch.max(outputs, 1)
                        
                        loss = criterion(outputs, labels)
                
                    running_loss += loss.item() * inputs.size(0)
                    running_corrects += torch.sum(preds == labels.data)
                
                epoch_loss = running_loss / test_size
                epoch_acc = running_corrects.double() / test_size
                
                print('{} Loss: {:.4f} Acc: {:.4f}'.format('Val', epoch_loss, epoch_acc))
                if epoch_acc > best_acc:
                    best_acc = epoch_acc
                    best_model_wts = copy.deepcopy(model.state_dict())

                last_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    
    return model, best_acc, last_model_wts

In [ ]:
net1 = Net()
net2 = Net()
net3 = Net()
net4 = Net()
net5 = Net()

In [ ]:
#---------------- Define a Loss function and optimizer -------------------

import torch.optim as optim

criterion = nn.CrossEntropyLoss()


In [ ]:
# Observe that all parameters are being optimized
optimizer1 = optim.Adam(net1.parameters(), lr=0.001, weight_decay=1e-07)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler_1 = lr_scheduler.StepLR(optimizer1, step_size=20, gamma=0.1)

# Observe that all parameters are being optimized
optimizer2 = optim.Adam(net2.parameters(), lr=0.001, weight_decay=1e-07)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler_2 = lr_scheduler.StepLR(optimizer2, step_size=20, gamma=0.1)

# Observe that all parameters are being optimized
optimizer3 = optim.Adam(net3.parameters(), lr=0.001, weight_decay=1e-07)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler_3 = lr_scheduler.StepLR(optimizer3, step_size=20, gamma=0.1)

# Observe that all parameters are being optimized
optimizer4 = optim.Adam(net4.parameters(), lr=0.001, weight_decay=1e-07)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler_4 = lr_scheduler.StepLR(optimizer4, step_size=20, gamma=0.1)

# Observe that all parameters are being optimized
optimizer5 = optim.Adam(net5.parameters(), lr=0.001, weight_decay=1e-07)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler_5 = lr_scheduler.StepLR(optimizer5, step_size=20, gamma=0.1)

In [ ]:
num_epochs = 30

In [ ]:
best_net1, best_acc1, last_net1 = train_model(net1, train_v_dl, test_dl, criterion, optimizer1, exp_lr_scheduler_1,
                        num_epochs=num_epochs)

Epoch 0/29
----------
tensor(15451)
Train Loss: 0.7302 Acc: 0.8829
Val Loss: 0.1163 Acc: 0.9634

Epoch 1/29
----------
tensor(17016)
Train Loss: 0.0902 Acc: 0.9723
Val Loss: 0.0920 Acc: 0.9696

Epoch 2/29
----------
tensor(17171)
Train Loss: 0.0586 Acc: 0.9812
Val Loss: 0.0953 Acc: 0.9763

Epoch 3/29
----------
tensor(17275)
Train Loss: 0.0408 Acc: 0.9871
Val Loss: 0.1006 Acc: 0.9746

Epoch 4/29
----------
tensor(17300)
Train Loss: 0.0360 Acc: 0.9886
Val Loss: 0.1089 Acc: 0.9756

Epoch 5/29
----------
tensor(17254)
Train Loss: 0.0417 Acc: 0.9859
Val Loss: 0.1023 Acc: 0.9760

Epoch 6/29
----------
tensor(17350)
Train Loss: 0.0265 Acc: 0.9914
Val Loss: 0.0976 Acc: 0.9796

Epoch 7/29
----------
tensor(17312)
Train Loss: 0.0310 Acc: 0.9893
Val Loss: 0.0927 Acc: 0.9785

Epoch 8/29
----------
tensor(17437)
Train Loss: 0.0108 Acc: 0.9964
Val Loss: 0.1065 Acc: 0.9794

Epoch 9/29
----------
tensor(17454)
Train Loss: 0.0079 Acc: 0.9974
Val Loss: 0.1235 Acc: 0.9771

Epoch 10/29
----------
tensor(

In [ ]:
best_net2, best_acc2, last_net2 = train_model(net2, train_w_dl, test_dl, criterion, optimizer2, exp_lr_scheduler_2,
                        num_epochs=num_epochs)

Epoch 0/29
----------
tensor(15561)
Train Loss: 0.7084 Acc: 0.8892
Val Loss: 0.1069 Acc: 0.9683

Epoch 1/29
----------
tensor(17009)
Train Loss: 0.0915 Acc: 0.9719
Val Loss: 0.1584 Acc: 0.9554

Epoch 2/29
----------
tensor(17117)
Train Loss: 0.0693 Acc: 0.9781
Val Loss: 0.1028 Acc: 0.9703

Epoch 3/29
----------
tensor(17269)
Train Loss: 0.0392 Acc: 0.9868
Val Loss: 0.1095 Acc: 0.9723

Epoch 4/29
----------
tensor(17274)
Train Loss: 0.0366 Acc: 0.9871
Val Loss: 0.0881 Acc: 0.9781

Epoch 5/29
----------
tensor(17363)
Train Loss: 0.0236 Acc: 0.9922
Val Loss: 0.0885 Acc: 0.9779

Epoch 6/29
----------
tensor(17330)
Train Loss: 0.0304 Acc: 0.9903
Val Loss: 0.1079 Acc: 0.9748

Epoch 7/29
----------
tensor(17355)
Train Loss: 0.0274 Acc: 0.9917
Val Loss: 0.1117 Acc: 0.9745

Epoch 8/29
----------
tensor(17345)
Train Loss: 0.0289 Acc: 0.9911
Val Loss: 0.1095 Acc: 0.9781

Epoch 9/29
----------
tensor(17363)
Train Loss: 0.0238 Acc: 0.9922
Val Loss: 0.1779 Acc: 0.9643

Epoch 10/29
----------
tensor(

In [ ]:
best_net5, best_acc5, last_net5 = train_model(net5, train_z_dl, test_dl, criterion, optimizer5, exp_lr_scheduler_5,
                        num_epochs=num_epochs)

Epoch 0/29
----------
tensor(15709)
Train Loss: 0.6780 Acc: 0.8977
Val Loss: 0.1199 Acc: 0.9650

Epoch 1/29
----------
tensor(17000)
Train Loss: 0.0912 Acc: 0.9714
Val Loss: 0.1085 Acc: 0.9673

Epoch 2/29
----------
tensor(17185)
Train Loss: 0.0547 Acc: 0.9820
Val Loss: 0.0970 Acc: 0.9734

Epoch 3/29
----------
tensor(17271)
Train Loss: 0.0387 Acc: 0.9869
Val Loss: 0.1148 Acc: 0.9697

Epoch 4/29
----------
tensor(17313)
Train Loss: 0.0307 Acc: 0.9893
Val Loss: 0.0901 Acc: 0.9777

Epoch 5/29
----------
tensor(17270)
Train Loss: 0.0402 Acc: 0.9869
Val Loss: 0.1126 Acc: 0.9747

Epoch 6/29
----------
tensor(17361)
Train Loss: 0.0245 Acc: 0.9921
Val Loss: 0.1118 Acc: 0.9773

Epoch 7/29
----------
tensor(17394)
Train Loss: 0.0203 Acc: 0.9939
Val Loss: 0.1169 Acc: 0.9772

Epoch 8/29
----------
tensor(17330)
Train Loss: 0.0324 Acc: 0.9903
Val Loss: 0.1013 Acc: 0.9766

Epoch 9/29
----------
tensor(17391)
Train Loss: 0.0202 Acc: 0.9938
Val Loss: 0.1124 Acc: 0.9786

Epoch 10/29
----------
tensor(

In [ ]:
best_net3, best_acc3, last_net3 = train_model(net3, train_x_dl, test_dl, criterion, optimizer3, exp_lr_scheduler_3,
                        num_epochs=num_epochs)

Epoch 0/29
----------
tensor(15679)
Train Loss: 0.6320 Acc: 0.8959
Val Loss: 0.1536 Acc: 0.9547

Epoch 1/29
----------
tensor(16983)
Train Loss: 0.0924 Acc: 0.9705
Val Loss: 0.1067 Acc: 0.9697

Epoch 2/29
----------
tensor(17176)
Train Loss: 0.0569 Acc: 0.9815
Val Loss: 0.0751 Acc: 0.9781

Epoch 3/29
----------
tensor(17274)
Train Loss: 0.0410 Acc: 0.9871
Val Loss: 0.1119 Acc: 0.9698

Epoch 4/29
----------
tensor(17273)
Train Loss: 0.0365 Acc: 0.9870
Val Loss: 0.1023 Acc: 0.9727

Epoch 5/29
----------
tensor(17327)
Train Loss: 0.0289 Acc: 0.9901
Val Loss: 0.1006 Acc: 0.9749

Epoch 6/29
----------
tensor(17368)
Train Loss: 0.0243 Acc: 0.9925
Val Loss: 0.1120 Acc: 0.9766

Epoch 7/29
----------
tensor(17374)
Train Loss: 0.0230 Acc: 0.9928
Val Loss: 0.1290 Acc: 0.9730

Epoch 8/29
----------
tensor(17354)
Train Loss: 0.0247 Acc: 0.9917
Val Loss: 0.1466 Acc: 0.9707

Epoch 9/29
----------
tensor(17338)
Train Loss: 0.0300 Acc: 0.9907
Val Loss: 0.1523 Acc: 0.9719

Epoch 10/29
----------
tensor(

In [ ]:
best_net4, best_acc4, last_net4 = train_model(net4, train_y_dl, test_dl, criterion, optimizer4, exp_lr_scheduler_4,
                        num_epochs=num_epochs)

Epoch 0/29
----------
tensor(15392)
Train Loss: 0.7295 Acc: 0.8795
Val Loss: 0.1514 Acc: 0.9509

Epoch 1/29
----------
tensor(16872)
Train Loss: 0.1127 Acc: 0.9641
Val Loss: 0.1160 Acc: 0.9669

Epoch 2/29
----------
tensor(17091)
Train Loss: 0.0755 Acc: 0.9766
Val Loss: 0.1120 Acc: 0.9647

Epoch 3/29
----------
tensor(17165)
Train Loss: 0.0566 Acc: 0.9809
Val Loss: 0.1122 Acc: 0.9683

Epoch 4/29
----------
tensor(17238)
Train Loss: 0.0478 Acc: 0.9850
Val Loss: 0.1031 Acc: 0.9709

Epoch 5/29
----------
tensor(17253)
Train Loss: 0.0409 Acc: 0.9859
Val Loss: 0.1236 Acc: 0.9703

Epoch 6/29
----------
tensor(17322)
Train Loss: 0.0305 Acc: 0.9898
Val Loss: 0.1174 Acc: 0.9735

Epoch 7/29
----------
tensor(17292)
Train Loss: 0.0324 Acc: 0.9881
Val Loss: 0.1434 Acc: 0.9694

Epoch 8/29
----------
tensor(17325)
Train Loss: 0.0298 Acc: 0.9900
Val Loss: 0.1189 Acc: 0.9747

Epoch 9/29
----------
tensor(17363)
Train Loss: 0.0247 Acc: 0.9922
Val Loss: 0.1039 Acc: 0.9804

Epoch 10/29
----------
tensor(

In [ ]:
correct_prob_1 = 0
total_prob_1 = 0
post_prob_1 = []
pred_prob_1 = []

epochs = 10
post_probs_1 = []

for epoch in range(epochs):
    with torch.no_grad():
        best_net1.eval()
        for data in prob_dl:
            images_prob_1, labels_prob_1 = data
            outputs_prob_1 = best_net1(images_prob_1)

            sm1 = torch.nn.Softmax()
            probabilities_prob_1 = sm1(outputs_prob_1)
            post_prob_1.append(probabilities_prob_1)
            _, predicted_prob_1 = torch.max(outputs_prob_1.data, 1)
            pred_prob_1.append(predicted_prob_1.item())
            

            total_prob_1 += labels_prob_1.size(0)
            correct_prob_1 += (predicted_prob_1== labels_prob_1).sum().item()
    accuracy = 100 * (correct_prob_1 / total_prob_1)
    post_probs_1.append(post_prob_1)

print('Probabilities', probabilities_prob_1)
print('length of Probabilities', len(post_prob_1))
print('Total Images: ', total_prob_1)
print('Accuracy of the network on the 11000 test images:',  accuracy)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Probabilities tensor([[2.1290e-25, 2.4396e-17, 1.0000e+00, 5.3378e-24, 2.9019e-23, 1.5574e-27,
         1.4317e-34, 3.2980e-25, 5.7248e-23, 4.6112e-25]])
length of Probabilities 50000
Total Images:  50000
Accuracy of the network on the 11000 test images: 98.56


In [ ]:
len(post_probs_1)

50000

In [ ]:
correct_prob_2 = 0
total_prob_2 = 0
post_prob_2 = []
pred_prob_2 = []

epochs = 10
post_probs_2 = []

for epoch in range(epochs):
    with torch.no_grad():
        best_net2.eval()
        for data in prob_dl:
            images_prob_2, labels_prob_2 = data
            outputs_prob_2 = best_net2(images_prob_2)

            sm2 = torch.nn.Softmax()
            probabilities_prob_2 = sm2(outputs_prob_2)
            post_prob_2.append(probabilities_prob_2)
            _, predicted_prob_2 = torch.max(outputs_prob_2.data, 1)
            pred_prob_2.append(predicted_prob_2.item())
            

            total_prob_2 += labels_prob_2.size(0)
            correct_prob_2 += (predicted_prob_2== labels_prob_2).sum().item()
    accuracy2 = 100 * (correct_prob_2 / total_prob_2)
    post_probs_2.append(post_prob_2)

print('Probabilities', probabilities_prob_2)
print('length of Probabilities', len(post_prob_2))
print('Total Images: ', total_prob_2)
print('Accuracy of the network on the 11000 test images:',  accuracy2)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Probabilities tensor([[1.4172e-15, 2.9050e-13, 1.0000e+00, 1.9719e-11, 2.3312e-13, 5.0118e-17,
         7.3261e-20, 6.4468e-13, 4.0697e-06, 2.3454e-12]])
length of Probabilities 50000
Total Images:  50000
Accuracy of the network on the 11000 test images: 98.72


In [ ]:
len(post_probs_2)

10

In [ ]:
correct_prob_3 = 0
total_prob_3 = 0
post_prob_3 = []
pred_prob_3 = []

epochs = 10
post_probs_3 = []

for epoch in range(epochs):
    with torch.no_grad():
        best_net3.eval()
        for data in prob_dl:
            images_prob_3, labels_prob_3 = data
            outputs_prob_3 = best_net3(images_prob_3)

            sm3 = torch.nn.Softmax()
            probabilities_prob_3 = sm3(outputs_prob_3)
            post_prob_3.append(probabilities_prob_3)
            _, predicted_prob_3 = torch.max(outputs_prob_3.data, 1)
            pred_prob_3.append(predicted_prob_3.item())
            

            total_prob_3 += labels_prob_3.size(0)
            correct_prob_3 += (predicted_prob_3 == labels_prob_3).sum().item()
    accuracy3 = 100 * (correct_prob_3 / total_prob_3)
    post_probs_3.append(post_prob_3)

print('Probabilities', probabilities_prob_3)
print('length of Probabilities', len(post_prob_3))
print('Total Images: ', total_prob_3)
print('Accuracy of the network on the 11000 test images:',  accuracy3)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Probabilities tensor([[2.2834e-17, 4.8617e-14, 1.0000e+00, 1.6915e-12, 4.1438e-22, 5.4676e-20,
         5.6909e-19, 2.3004e-17, 7.4926e-12, 3.8881e-13]])
length of Probabilities 50000
Total Images:  50000
Accuracy of the network on the 11000 test images: 98.72


In [ ]:
len(post_probs_3)

10

In [ ]:
correct_prob_4 = 0
total_prob_4 = 0
post_prob_4 = []
pred_prob_4 = []

epochs = 10
post_probs_4 = []

for epoch in range(epochs):
    with torch.no_grad():
        best_net4.eval()
        for data in prob_dl:
            images_prob_4, labels_prob_4 = data
            outputs_prob_4 = best_net4(images_prob_4)

            sm4 = torch.nn.Softmax()
            probabilities_prob_4 = sm4(outputs_prob_4)
            post_prob_4.append(probabilities_prob_4)
            _, predicted_prob_4 = torch.max(outputs_prob_4.data, 1)
            pred_prob_4.append(predicted_prob_4.item())
            

            total_prob_4 += labels_prob_4.size(0)
            correct_prob_4 += (predicted_prob_4 == labels_prob_4).sum().item()
    accuracy4 = 100 * (correct_prob_4 / total_prob_4)
    post_probs_4.append(post_prob_4)

print('Probabilities', probabilities_prob_4)
print('length of Probabilities', len(post_prob_4))
print('Total Images: ', total_prob_4)
print('Accuracy of the network on the 11000 test images:',  accuracy4)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Probabilities tensor([[4.0795e-16, 1.5751e-07, 1.0000e+00, 3.5783e-12, 2.7458e-17, 1.4462e-19,
         7.5041e-17, 1.2362e-13, 5.7614e-13, 1.3550e-12]])
length of Probabilities 50000
Total Images:  50000
Accuracy of the network on the 11000 test images: 98.32


In [ ]:
len(post_probs_4)

10

In [ ]:
correct_prob_5 = 0
total_prob_5 = 0
post_prob_5 = []
pred_prob_5 = []

epochs = 10
post_probs_5 = []

for epoch in range(epochs):
    with torch.no_grad():
        best_net5.eval()
        for data in prob_dl:
            images_prob_5, labels_prob_5 = data
            outputs_prob_5 = best_net5(images_prob_5)

            sm5 = torch.nn.Softmax()
            probabilities_prob_5 = sm5(outputs_prob_5)
            post_prob_5.append(probabilities_prob_5)
            _, predicted_prob_5 = torch.max(outputs_prob_5.data, 1)
            pred_prob_5.append(predicted_prob_5.item())
            

            total_prob_5 += labels_prob_5.size(0)
            correct_prob_5 += (predicted_prob_5 == labels_prob_5).sum().item()
        accuracy5 = 100 * (correct_prob_5 / total_prob_5)
        post_probs_5.append(post_prob_5)

print('Probabilities', probabilities_prob_5)
print('length of Probabilities', len(post_prob_5))
print('Total Images: ', total_prob_5)
print('Accuracy of the network on the 11000 test images:',  accuracy5)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Probabilities tensor([[1.7856e-22, 2.6034e-11, 1.0000e+00, 5.5756e-12, 6.2356e-19, 1.5053e-21,
         3.2385e-20, 5.4753e-18, 3.5653e-18, 2.8973e-18]])
length of Probabilities 50000
Total Images:  50000
Accuracy of the network on the 11000 test images: 98.46000000000001


In [ ]:
len(post_probs_5)
post_probs_5[0][2] == post_probs_5[1][2]

tensor([[True, True, True, True, True, True, True, True, True, True]])

In [ ]:
# Calculate MSE for any two set of posterior probabilities that observed from two different models

def nn_mse_loss(post_1, post_2):
  mse_loss_meth = nn.MSELoss()
  mse_loss = mse_loss_meth(torch.stack(post_1), torch.stack(post_2))
  return mse_loss.item()

In [ ]:
mse_losses_1_2 = []

for l1, l2 in zip(post_probs_1, post_probs_2):
    mse_losses_1_2.append(nn_mse_loss(l1, l2))
print(mse_losses_1_2)
    

[0.0029438864439725876, 0.0029438864439725876, 0.0029438864439725876, 0.0029438864439725876, 0.0029438864439725876, 0.0029438864439725876, 0.0029438864439725876, 0.0029438864439725876, 0.0029438864439725876, 0.0029438864439725876]


In [ ]:
mse_losses_1_3 = []

for l1, l3 in zip(post_probs_1, post_probs_3):
    mse_losses_1_3.append(nn_mse_loss(l1, l3))
print(mse_losses_1_3)

[0.002516572130843997, 0.002516572130843997, 0.002516572130843997, 0.002516572130843997, 0.002516572130843997, 0.002516572130843997, 0.002516572130843997, 0.002516572130843997, 0.002516572130843997, 0.002516572130843997]


In [ ]:
mse_losses_1_4 = []

for l1, l4 in zip(post_probs_1, post_probs_4):
    mse_losses_1_4.append(nn_mse_loss(l1, l4))
print(mse_losses_1_4)

[0.0029750007670372725, 0.0029750007670372725, 0.0029750007670372725, 0.0029750007670372725, 0.0029750007670372725, 0.0029750007670372725, 0.0029750007670372725, 0.0029750007670372725, 0.0029750007670372725, 0.0029750007670372725]


In [ ]:
mse_losses_1_5 = []

for l1, l5 in zip(post_probs_1, post_probs_5):
    mse_losses_1_5.append(nn_mse_loss(l1, l5))
print(mse_losses_1_5)

[0.0024781536776572466, 0.0024781536776572466, 0.0024781536776572466, 0.0024781536776572466, 0.0024781536776572466, 0.0024781536776572466, 0.0024781536776572466, 0.0024781536776572466, 0.0024781536776572466, 0.0024781536776572466]


In [ ]:
#Calculate Model Similarity

def model_similarity(mod_1, mod_2):
  zip_lists = zip(mod_1, mod_2)
  corr = 0
  total = 0
  for list_t, list_p in zip_lists:
    total += 1
    if list_t-list_p == 0:
      corr += 1

  accuracy = (corr / total) * 100

  print("Model Similarity: ", accuracy, "%")

In [ ]:
# Print the model similarity

model_similarity(pred_prob_1, pred_prob_2)
model_similarity(pred_prob_1, pred_prob_3)
model_similarity(pred_prob_1, pred_prob_4)
model_similarity(pred_prob_1, pred_prob_5)

Model Similarity:  98.16 %
Model Similarity:  98.5 %
Model Similarity:  98.26 %
Model Similarity:  98.34 %


In [ ]:
from sklearn.metrics import *
from matplotlib import pyplot as plt

In [ ]:

def arrange_mses(all_mses):
    count_mse = 0
    mse_data = []
    mse_target = []

    for mse_l in all_mses:
        for mse in mse_l:
            count_mse += 1
            if count_mse <= 10:
                mse_data.append(mse)
                mse_target.append(1)
            else:
                mse_data.append(mse)
                mse_target.append(0)
    
    mse_dict = {'data': mse_data,
                'target': mse_target
                }

    return mse_dict





In [ ]:
all_mses1 = [mse_losses_1_2, mse_losses_1_3, mse_losses_1_4, mse_losses_1_5]

mses1 = arrange_mses(all_mses1)

mse_df1 = pd.DataFrame(mses1, columns = ['data','target'], index=['mse_1_2', 'mse_1_2', 'mse_1_2', 'mse_1_2', 'mse_1_2', 'mse_1_2', 'mse_1_2', 'mse_1_2', 'mse_1_2', 'mse_1_2', 'mse_1_3', 'mse_1_3', 'mse_1_3', 'mse_1_3', 'mse_1_3', 'mse_1_3', 'mse_1_3', 'mse_1_3', 'mse_1_3', 'mse_1_3', 'mse_1_4', 'mse_1_4', 'mse_1_4', 'mse_1_4', 'mse_1_4', 'mse_1_4', 'mse_1_4', 'mse_1_4', 'mse_1_4', 'mse_1_4', 'mse_1_5', 'mse_1_5', 'mse_1_5', 'mse_1_5', 'mse_1_5', 'mse_1_5', 'mse_1_5', 'mse_1_5', 'mse_1_5', 'mse_1_5'])

mse_df1 = mse_df1.sort_values(by=['data'])

#print the labeled mses
print (mse_df1.head(5))

             data  target
mse_1_5  0.002478       0
mse_1_5  0.002478       0
mse_1_5  0.002478       0
mse_1_5  0.002478       0
mse_1_5  0.002478       0


In [ ]:
all_mses2 = [mse_losses_1_3, mse_losses_1_2, mse_losses_1_4, mse_losses_1_5]

mses2 = arrange_mses(all_mses2)

mse_df2 = pd.DataFrame(mses2, columns = ['data','target'], index=['mse_1_3', 'mse_1_3', 'mse_1_3', 'mse_1_3', 'mse_1_3', 'mse_1_3', 'mse_1_3', 'mse_1_3', 'mse_1_3', 'mse_1_3', 'mse_1_2', 'mse_1_2', 'mse_1_2', 'mse_1_2', 'mse_1_2', 'mse_1_2', 'mse_1_2', 'mse_1_2', 'mse_1_2', 'mse_1_2',  'mse_1_4', 'mse_1_4', 'mse_1_4', 'mse_1_4', 'mse_1_4', 'mse_1_4', 'mse_1_4', 'mse_1_4', 'mse_1_4', 'mse_1_4', 'mse_1_5', 'mse_1_5', 'mse_1_5', 'mse_1_5', 'mse_1_5', 'mse_1_5', 'mse_1_5', 'mse_1_5', 'mse_1_5', 'mse_1_5'])

mse_df2 = mse_df2.sort_values(by=['data'])

#print the labeled mses
print (mse_df2.head(5))

             data  target
mse_1_5  0.002478       0
mse_1_5  0.002478       0
mse_1_5  0.002478       0
mse_1_5  0.002478       0
mse_1_5  0.002478       0


In [ ]:
all_mses3 = [mse_losses_1_4, mse_losses_1_2, mse_losses_1_3, mse_losses_1_5]

mses3 = arrange_mses(all_mses3)

mse_df3 = pd.DataFrame(mses3, columns = ['data','target'], index=['mse_1_4', 'mse_1_4', 'mse_1_4', 'mse_1_4', 'mse_1_4', 'mse_1_4', 'mse_1_4', 'mse_1_4', 'mse_1_4', 'mse_1_4', 'mse_1_2', 'mse_1_2', 'mse_1_2', 'mse_1_2', 'mse_1_2', 'mse_1_2', 'mse_1_2', 'mse_1_2', 'mse_1_2', 'mse_1_2', 'mse_1_3', 'mse_1_3', 'mse_1_3', 'mse_1_3', 'mse_1_3', 'mse_1_3', 'mse_1_3', 'mse_1_3', 'mse_1_3', 'mse_1_3', 'mse_1_5', 'mse_1_5', 'mse_1_5', 'mse_1_5', 'mse_1_5', 'mse_1_5', 'mse_1_5', 'mse_1_5', 'mse_1_5', 'mse_1_5'])

mse_df3 = mse_df3.sort_values(by=['data'])

#print the labeled mses
print (mse_df3.head(5))

             data  target
mse_1_5  0.002478       0
mse_1_5  0.002478       0
mse_1_5  0.002478       0
mse_1_5  0.002478       0
mse_1_5  0.002478       0


In [ ]:
all_mses4 = [mse_losses_1_5, mse_losses_1_2, mse_losses_1_3, mse_losses_1_4]

mses4 = arrange_mses(all_mses4)

mse_df4 = pd.DataFrame(mses4, columns = ['data','target'], index=['mse_1_5', 'mse_1_5', 'mse_1_5', 'mse_1_5', 'mse_1_5', 'mse_1_5', 'mse_1_5', 'mse_1_5', 'mse_1_5', 'mse_1_2', 'mse_1_2', 'mse_1_2', 'mse_1_2', 'mse_1_2', 'mse_1_2', 'mse_1_2', 'mse_1_2', 'mse_1_2', 'mse_1_2', 'mse_1_3', 'mse_1_3', 'mse_1_3', 'mse_1_3', 'mse_1_3', 'mse_1_3', 'mse_1_3', 'mse_1_3', 'mse_1_3', 'mse_1_3', 'mse_1_4', 'mse_1_4', 'mse_1_4', 'mse_1_4', 'mse_1_4', 'mse_1_4', 'mse_1_4', 'mse_1_4', 'mse_1_4', 'mse_1_4', 'mse_1_5'])

mse_df4 = mse_df4.sort_values(by=['data'])

#print the labeled mses
print (mse_df4.head(5))

             data  target
mse_1_5  0.002478       1
mse_1_5  0.002478       1
mse_1_5  0.002478       1
mse_1_5  0.002478       1
mse_1_5  0.002478       1


In [ ]:
#calculate the auc

mses_auc1 = auc(mse_df1.data, mse_df1.target)

In [ ]:
mses_auc1

0.00022921431809663773

In [ ]:
#calculate the auc

mses_auc2 = auc(mse_df2.data, mse_df2.target)

In [ ]:
mses_auc2

0.0002328663831576705

In [ ]:
#calculate the auc

mses_auc3 = auc(mse_df3.data, mse_df3.target)

In [ ]:
mses_auc3

1.5557161532342434e-05

In [ ]:
#calculate the auc

mses_auc4 = auc(mse_df4.data, mse_df4.target)

In [ ]:
mses_auc4

1.9209226593375206e-05

In [ ]:
MSE_AUC1 = 1 - mses_auc1
MSE_AUC2 = 1 - mses_auc2
MSE_AUC3 = 1 - mses_auc3
MSE_AUC4 = 1 - mses_auc4


In [ ]:
print("MSE_AUC1: ", MSE_AUC1)
print("MSE_AUC2: ", MSE_AUC2)
print("MSE_AUC3: ", MSE_AUC3)
print("MSE_AUC4: ", MSE_AUC4)

MSE_AUC1:  0.9997707856819034
MSE_AUC2:  0.9997671336168423
MSE_AUC3:  0.9999844428384677
MSE_AUC4:  0.9999807907734066
